
# Rank ordered logFC*expression on Pancreas training Dataset

In [2]:
!pip install scanpy -q

In [3]:
import numpy as np
import pandas as  pd
import scanpy as sc
from sklearn.linear_model import LinearRegression


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# TRAINING DATA

## Read the Training data

In [5]:
## Read train data
# adata_train = sc.read('/content/gdrive/MyDrive/Shared resources/Bh.h5ad')
adata_train = sc.read('/content/gdrive/MyDrive/Shared resources/Baron_pancreatic_islet.h5ad')
adata_train

AnnData object with n_obs × n_vars = 8569 × 20125
    obs: 'celltype'

## Preprocess the train data

In [6]:
adata_train.obs_names_make_unique()
sc.pp.filter_cells(adata_train, min_genes=200)
sc.pp.filter_genes(adata_train, min_cells=3)
sc.pp.normalize_total(adata_train, target_sum=1e4)
sc.pp.log1p(adata_train)
#sc.pp.highly_variable_genes(adata_train, n_top_genes = 1000)
adata_train.raw = adata_train
#adata_train = adata_train[:, adata_train.var.highly_variable]
sc.pp.scale(adata_train, max_value=10)
adata_train.shape

(8569, 16359)

# Return logFC*expression train matrix


In [7]:
def rank_mult_logFC_expr_train(adata_train, groupby_col, top_de_genes = 50):
  """
  This function find the top n upregulated and downregulated genes and returns the ranked logFC*expression matrix
  - We can do this step before or after dropout induction
  - We will select both upregulated and downregulated genes
  INPUT:
      data: anndata containing cell-gene expression matrix
      groupby_col: key of the observations grouping
      top_de_genes: number of top upregulated and downregulated de genes

  OUTPUT:
      adata: transformed anndata with rank ordered logFC*expression matrix
      adata.obs: transformed observations grouping
  """
  sc.tl.rank_genes_groups(adata_train, groupby = groupby_col, method='wilcoxon', use_raw = True)
  result = adata_train.uns['rank_genes_groups']
  groups = result['names'].dtype.names
  # Matrix sorted by logfoldchange
  detrain_dict = {}
  for group in groups:
      gene_rank_df = sc.get.rank_genes_groups_df(adata_train, group=group, pval_cutoff=0.05)
      print("Number of DE genes in {} = {}".format(group,len(gene_rank_df)))
      gene_rank_df.sort_values(by=['logfoldchanges'], inplace=True, ascending=False)
      if len(gene_rank_df) < 50:
          lfc_genes_df = gene_rank_df
      if len(gene_rank_df) >= 50:
          upregulated_genes = gene_rank_df.head(50)
          dnregulated_genes = gene_rank_df.tail(50)
          lfc_genes_df = pd.concat([upregulated_genes, dnregulated_genes], axis=0)
      detrain_dict[group] = dict(zip(lfc_genes_df['names'], lfc_genes_df['logfoldchanges']))

  # Take all the DE genes to create subset of genes in the main matrix 
  tot_gene_list = list(set([key for subdict in detrain_dict.values() for key in subdict.keys()]))
  print("Number of unique DE genes across all groups = {}".format(len(tot_gene_list)))
  
  # select only the subset of columns in the obs dataframe
  adata_sub = adata_train[:,tot_gene_list]

  # For each of the groups multiply the DE genes with the logFC with the expression
  sub_list = []
  for group in groups:
      gdata = adata_sub[adata_sub.obs[groupby_col] == group, :].to_df()
      for gene, factor in detrain_dict[group].items():
          gdata[gene] = gdata[gene]* abs(factor)
      gdata = gdata.assign(celltype=group)
      sub_list.append(gdata)

  del adata_sub, detrain_dict

  transformed_count  = pd.concat(sub_list, axis=0)
  transformed_group = transformed_count[[groupby_col]]
  # rank the values in each row
  df_ranked = transformed_count.drop(groupby_col, axis=1).rank(axis=1, method='min', ascending=False).astype(int)

  # create anndata for ranked tranformed matrix
  adata = sc.AnnData(df_ranked)
  adata.obs[groupby_col] = transformed_group
  
  del transformed_count, df_ranked
  return adata

In [8]:
transform_adata_train = rank_mult_logFC_expr_train(adata_train, "celltype", 50)

Number of DE genes in acinar = 4897
Number of DE genes in activated_stellate = 3370
Number of DE genes in alpha = 4381
Number of DE genes in beta = 5051
Number of DE genes in delta = 2135
Number of DE genes in ductal = 5021
Number of DE genes in endothelial = 2561
Number of DE genes in epsilon = 34
Number of DE genes in gamma = 1118
Number of DE genes in macrophage = 1100
Number of DE genes in mast = 350
Number of DE genes in quiescent_stellate = 1635
Number of DE genes in schwann = 59
Number of DE genes in t_cell = 3
Number of unique DE genes across all groups = 983


In [9]:
transform_adata_train

AnnData object with n_obs × n_vars = 8569 × 983
    obs: 'celltype'

In [ ]:
transform_adata_train.to_df()

index,LSAMP,PCSK1,ESAM,KIAA0319,RAMP2,CAPG,SLC45A3,PPY2P,REST,S100A13,...,CD4,MECOM,ERBB3,CALCB,HSD17B2,NEFM,DUSP26,CD300LB,EPS8L3,HSD11B1
index,,,,,,,,,,,,,,,,,,,,,
human1_lib1.final_cell_0001,750,876,544,976,595,58,642,295,641,105,...,365,494,71,410,447,652,946,181,279,348
human1_lib1.final_cell_0002,772,883,578,976,628,730,668,329,667,71,...,397,528,718,441,479,677,946,211,312,380
human1_lib1.final_cell_0003,708,852,486,976,540,54,585,243,584,773,...,311,439,644,355,392,595,946,127,228,294
human1_lib1.final_cell_0004,746,859,546,976,598,700,644,301,643,800,...,369,498,688,413,451,651,946,189,286,352
human1_lib1.final_cell_0005,732,862,526,976,578,81,621,277,52,65,...,345,476,74,390,428,632,946,163,262,328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
human2_lib2.final_cell_0582,702,885,426,575,491,20,554,178,553,788,...,247,375,624,290,327,567,856,65,162,229
human2_lib2.final_cell_0590,697,881,418,568,482,17,547,172,546,784,...,242,368,617,284,319,560,854,62,157,224
human3_lib3.final_cell_0866,694,875,419,567,483,15,546,174,545,780,...,241,370,616,284,321,559,847,62,158,223


# Read Test data

In [10]:
## Read test data
# adata_test = sc.read('/content/gdrive/MyDrive/Shared resources/smartseq2.h5ad')
adata_test = sc.read('/content/gdrive/MyDrive/Shared resources/Segerstolpe_pancreatic_islet.h5ad')
adata_test

AnnData object with n_obs × n_vars = 2394 × 34363
    obs: 'celltype', 'tech'
    var: 'genename'

In [11]:
adata_test.obs_names_make_unique()
sc.pp.filter_cells(adata_test, min_genes=200)
sc.pp.filter_genes(adata_test, min_cells=3)
sc.pp.normalize_total(adata_test, target_sum=1e4)
sc.pp.log1p(adata_test)
#sc.pp.highly_variable_genes(adata_train, n_top_genes = 1000)
adata_test.raw = adata_test
#adata_train = adata_train[:, adata_train.var.highly_variable]
sc.pp.scale(adata_test, max_value=10)
adata_test.shape

(2394, 21117)

# Return rank ordered test matrix

In [12]:
def rank_expr_test(adata_test, gene_list):
  """
  This function find the common genes between train and test dataset. The output will be a rank ordered matrix.
  INPUT:
      data: anndata containing cell-gene expression matrix
      gene_list: gene sets from train data, adata_train.var_names

  OUTPUT:
      adata: transformed anndata with rank ordered  matrix
      adata.obs: transformed observations from original test anndata
  """
  # find the common genes between train and test dataset
  common_genes = gene_list.intersection(adata_test.var_names)
  print("Number of common genes of the reference and query sets = {}".format(len(common_genes)))
  
  # slice the test data based on common genes
  adata_test_aligned = adata_test[:, common_genes].copy()

  # rank order the test data
  df_ranked = adata_test_aligned.to_df().rank(axis=1, method='min', ascending=False).astype(int)

  # create anndata for ranked tranformed matrix
  adata = sc.AnnData(df_ranked)
  adata.obs = adata_test_aligned.obs

  del adata_test_aligned, df_ranked
  return adata
  


In [13]:
transform_adata_test = rank_expr_test(adata_test, transform_adata_train.var_names)

Number of common genes of the reference and query sets = 963


In [14]:
transform_adata_test

AnnData object with n_obs × n_vars = 2394 × 963
    obs: 'celltype', 'tech', 'n_genes'

In [15]:
transform_adata_test.to_df()

index,SLC34A2,C1orf106,CLDN3,RNF128,CHRM3,SMIM5,DPP6,MRC2,ID1,SYNDIG1,...,PODXL,TMC6,B3GNT8,ITGB6,BEX1,RCAN1,SPINK2,SDC4,TSLP,LAD1
index,,,,,,,,,,,,,,,,,,,,,
AZ_A2,582,636,939,46,123,2,920,383,812,706,...,495,674,751,671,102,739,384,841,202,794
AZ_H5,564,621,931,915,560,646,40,365,790,689,...,127,658,735,656,59,721,366,823,182,770
AZ_G5,598,647,940,137,595,6,35,399,819,717,...,221,685,764,683,91,749,400,850,215,800
AZ_D8,631,686,611,940,78,709,118,444,845,753,...,124,722,262,720,161,784,445,876,260,827
AZ_D12,585,642,104,933,581,669,926,387,491,711,...,499,682,42,679,115,741,388,842,200,794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HP1504901_A22,513,567,896,878,509,595,874,345,737,636,...,435,606,678,604,943,48,346,770,174,717
HP1504901_M11,501,557,890,873,497,585,870,326,728,626,...,419,596,669,594,946,656,327,762,159,707
HP1504901_N21,517,572,895,877,513,598,871,355,96,641,...,443,610,688,608,948,49,356,129,202,723


---
# **Implementation of DL classifier**

---


**@anunay you may continue from here**

train anndata = transform_adata_train

test anndata = transform_adata_test

In [16]:
train_df = transform_adata_train.to_df()
test_df = transform_adata_test.to_df()

In [17]:
y_train = transform_adata_train.obs.celltype.to_list()
y_test = transform_adata_test.obs.celltype.to_list()

In [18]:
labels = set(y_train)
mapping = {}
cnt = 0
for lab in set(y_train):
  if lab in mapping:
    continue
  mapping[lab] = cnt
  cnt += 1


In [19]:
y_test_lab = []
for i in y_test:
  if i in mapping:
    y_test_lab.append(mapping[i])
  else:
    y_test_lab.append(0)
y_test = np.array(y_test_lab)

In [20]:
y_train_lab = []
for i in y_train:
  if i in mapping:
    y_train_lab.append(mapping[i])
  else:
    y_train_lab.append(0)
y_train = np.array(y_train_lab)

In [21]:
print("Taking common genes...")
final_columns = list(set(train_df.columns).intersection(set(test_df.columns)))
print('Common columns', len(final_columns))
final_columns = [i for i in final_columns if i != 'celltype'] 
train_df = train_df[final_columns]
test_df = test_df[final_columns]

Taking common genes...
Common columns 963


In [22]:
X_train = train_df.to_numpy()
X_test = test_df.to_numpy()

# Keras tuner

In [23]:
!pip3 install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 7.7 MB/s eta 0:00:00


In [24]:
y_test_cat = np.zeros((y_test.shape[0], 14))
y_train_cat = np.zeros((y_train.shape[0], 14))

In [25]:
def convert_to_cat(y_new, y):
  for i in range(y.shape[0]):
    y_new[i, y[i]] = 1
  return y_new
y_test_cat = convert_to_cat(y_test_cat, y_test)
y_train_cat = convert_to_cat(y_train_cat, y_train)

In [26]:
X_train.shape

(8569, 963)

In [28]:
import numpy as np
import pandas as pd
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras

tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
np.random.seed(2)

##loading 90% Training data
directory = '/content/gdrive/MyDrive/Shared resources/'
project_name = 'RO_dataset_hyperparameter_tuning'


##Definining hyper parameters 
layers_range = (3, 6)
units_range = (128, 896, 128)
lr_values = [1e-4,3e-5,1e-5, 3e-6]

##Define model
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(units = 1024,input_dim = 963, activation = 'relu'))

  for i in range(hp.Int('layers', layers_range[0], layers_range[1])):
    model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),  
                                min_value=units_range[0], max_value=units_range[1], 
                                step=units_range[2]), activation='relu'))

    
    model.add(keras.layers.Dense(14, activation='softmax'))
    hp_learning_rate = hp.Choice('learning_rate', values=lr_values)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.CategoricalAccuracy()]) 
  return model

##Perform hyperparameter tuning
for i in range(5):
    offset = (X_train.shape[0]*9)//10
    tuner = kt.Hyperband(model_builder, # the hypermodel
                    objective='val_loss', # objective to optimize
                    max_epochs=256,
                    factor=4, 
                    directory=directory, # directory to save logs 
                    project_name=project_name+str(i+1))
    
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    tuner.search(X_train, y_train_cat, epochs=400, batch_size = 4096, validation_data = (X_test, y_test_cat), callbacks=[stop_early])
    best_hp=tuner.get_best_hyperparameters()[0]
    best_model = tuner.get_best_models()[0]
    
    # Build the model with the optimal hyperparameters
    h_model = tuner.hypermodel.build(best_hp)
    h_model.fit(X_train, y_train_cat, epochs=400, verbose = 1, batch_size = 4096, validation_data = (X_test, y_test_cat))
    h_model.save('precily_cv_'+str(i+1)+'.hdf5')
    h_model = None
    


Trial 64 Complete [00h 00m 05s]
val_loss: 2.627977132797241

Best val_loss So Far: 2.6243536472320557
Total elapsed time: 00h 06m 21s

Search: Running Trial #65

Value             |Best Value So Far |Hyperparameter
4                 |4                 |layers
768               |768               |units_0
3e-05             |0.0001            |learning_rate
256               |768               |units_1
768               |768               |units_2
640               |384               |units_3
256               |640               |units_4
128               |None              |units_5
1                 |1                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round



KeyboardInterrupt: ignored

In [37]:
model = keras.Sequential()
model.add(keras.layers.Dense(units = 512,input_dim = 963, activation = 'relu'))

# for i in range(4):
#   model.add(keras.layers.Dense(units=512, activation='relu'))
# model.add(keras.layers.Dense(units=512, activation='relu'))
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(14, activation='softmax'))
hp_learning_rate = 1e-5

model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.CategoricalAccuracy()]) 

In [ ]:
model.fit(X_train, y_train_cat, epochs=1000, verbose = 1, batch_size = 4096, validation_data = (X_test, y_test_cat))

Epoch 1/1000
3/3 [==============================] - 3s 557ms/step - loss: 920.1733 - auc_4: 0.4778 - categorical_accuracy: 0.0303 - val_loss: 736.7405 - val_auc_4: 0.5078 - val_categorical_accuracy: 0.0856
Epoch 2/1000
3/3 [==============================] - 1s 341ms/step - loss: 805.1699 - auc_4: 0.4782 - categorical_accuracy: 0.0315 - val_loss: 642.9548 - val_auc_4: 0.5038 - val_categorical_accuracy: 0.0789
Epoch 3/1000
3/3 [==============================] - 1s 321ms/step - loss: 695.7074 - auc_4: 0.4791 - categorical_accuracy: 0.0333 - val_loss: 557.7119 - val_auc_4: 0.5038 - val_categorical_accuracy: 0.0802
Epoch 4/1000
3/3 [==============================] - 1s 313ms/step - loss: 592.9579 - auc_4: 0.4810 - categorical_accuracy: 0.0368 - val_loss: 481.7903 - val_auc_4: 0.5049 - val_categorical_accuracy: 0.0806
Epoch 5/1000
3/3 [==============================] - 1s 297ms/step - loss: 497.6340 - auc_4: 0.4859 - categorical_accuracy: 0.0446 - val_loss: 415.2498 - val_auc_4: 0.5150 - val

KeyboardInterrupt: ignored

# NF


In [ ]:
import pickle

In [ ]:
with open('/content/flow_ssl/data/y_test.pkl', 'wb') as fh:
   pickle.dump(y_test, fh)
with open('/content/flow_ssl/data/y_train.pkl', 'wb') as fh:
   pickle.dump(y_train, fh)

In [ ]:
with open('/content/flow_ssl/data/X_test.pkl', 'wb') as fh:
   pickle.dump(X_test, fh)
with open('/content/flow_ssl/data/X_train.pkl', 'wb') as fh:
   pickle.dump(X_train, fh)

In [ ]:
!unzip "/content/gdrive/MyDrive/Shared resources/flowgmm.zip"

In [ ]:
!pip install -e .
!pip install timm==0.3.2 torch==1.8.1 torchvision 
!pip install scanpy normflows
!pip install pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/mfinzi/olive-oil-ml to /tmp/pip-install-3gif8w_0/olive-oil-ml_11cc7d9c8282430f8996302a8a5a14d0
  Running command git clone --filter=blob:none --quiet https://github.com/mfinzi/olive-oil-ml /tmp/pip-install-3gif8w_0/olive-oil-ml_11cc7d9c8282430f8996302a8a5a14d0
  Resolved https://github.com/mfinzi/olive-oil-ml to commit 32978a77414575fad65b916e75374f9e16e99ede
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Attempting uninstall: FlowGMM
    Found existing installation: FlowGMM 0.1
    Can't uninstall 'FlowGMM'. No files were found to uninstall.
  Running setup.py develop for FlowGMM
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-pytho

In [ ]:
!python3 experiments/train_flows/flowgmm_tabular_new.py --trainer_config "{'unlab_weight':.6}" --net_config "{'k':1024,'coupling_layers':7,'nperlayer':1}" --network RealNVPTabularWPrior --trainer SemiFlow --num_epochs 10000 --dataset AG_News --lr 3e-4 --train 10000 --bs 10000

{'dataset': <class 'flow_ssl.data.nlp_datasets.AG_News'>, 'network': <function RealNVPTabularWPrior at 0x7fc05038caf0>, 'num_epochs': 10000, 'bs': 10000, 'lr': 0.0003, 'optim': <class 'torch.optim.adamw.AdamW'>, 'device': 'cuda', 'trainer': SemiFlow, 'split': {'train': 10000, 'val': 5000}, 'net_config': {'k': 1024, 'coupling_layers': 7, 'nperlayer': 1}, 'opt_config': {'weight_decay': 1e-05}, 'trainer_config': {'log_dir': '/root/tb-experiments/UCI/', 'log_args': {'minPeriod': 0.1, 'timeFrac': 0.3}, 'unlab_weight': 0.6}, 'save': False}
8569
8569
2394
Pairwise dists: [[ 0.         23.98605122 24.8773274  23.68513762 25.41487821 25.20550683
  24.96208074 24.57989645 24.65671609 24.65561466 24.76541649 23.86222525
  25.2779631  25.14839989]
 [23.98605122  0.         24.96672304 24.46091736 24.58927195 24.92435047
  24.69337564 24.49599666 24.97331937 23.87492547 24.70093235 25.25464703
  24.68266664 25.50999955]
 [24.8773274  24.96672304  0.         24.16324134 24.26854227 24.25711302
  24.